1. Cargar y preprocesar el dataset

In [1]:
import os
import json
from PIL import Image

# Cargar los datos del dataset
base_path = "./"
with open(f'{base_path}/MMHS150K_GT.json', 'r') as f:
    data = json.load(f)

with open(f'{base_path}/splits/train_ids.txt', 'r') as f:
    id_train = f.read().split()
with open(f'{base_path}/splits/val_ids.txt', 'r') as f:
    id_val = f.read().split()
with open(f'{base_path}/splits/test_ids.txt', 'r') as f:
    id_test = f.read().split()

dict_data = {x: data[x] for x in id_train + id_val + id_test if x in data}

# Preprocesar y guardar el dataset
def process_and_save_MMHS150K(dict_data, img_dir, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    image_subfolder = os.path.join(output_folder, 'images')
    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)
    
    json_data_list = []

    total_items = len(dict_data)
    print(f"Total items to process: {total_items}")
    
    for count, (tweet_id, tweet_data) in enumerate(dict_data.items(), start=1):
        img_filename = tweet_id + ".jpg"
        img_path = os.path.join(img_dir, img_filename)
        
        # Asegurarse de que la imagen existe
        if not os.path.exists(img_path):
            continue
        
        # Verificar que el campo 'tweet_text' existe en tweet_data
        if 'tweet_text' not in tweet_data:
            continue

        image = Image.open(img_path).convert('RGB')
        unique_id = tweet_id
        image.save(os.path.join(image_subfolder, f"{unique_id}.jpg"))

        tweet_text = tweet_data['tweet_text']

        json_data = {
            "id": unique_id,
            "image": f"{unique_id}.jpg",
            "conversations": [
                {
                    "from": "human",
                    "value": "Generate a tweet for this meme image"
                },
                {
                    "from": "gpt",
                    "value": tweet_text
                }
            ]
        }

        json_data_list.append(json_data)

        # Imprimir cada 10 elementos procesados
        if count % 10 == 0:
            print(f"Processed {count}/{total_items} items")

    with open(os.path.join(output_folder, 'dataset.json'), 'w') as json_file:
        json.dump(json_data_list, json_file, indent=4)
    
    print("End of process")

process_and_save_MMHS150K(dict_data, base_path + 'img_resized', './dataset/train')


Total items to process: 149823
Processed 10/149823 items
Processed 20/149823 items
Processed 30/149823 items
Processed 40/149823 items
Processed 50/149823 items
Processed 60/149823 items
Processed 70/149823 items
Processed 80/149823 items
Processed 90/149823 items
Processed 100/149823 items
Processed 110/149823 items
Processed 120/149823 items
Processed 130/149823 items
Processed 140/149823 items
Processed 150/149823 items
Processed 160/149823 items
Processed 170/149823 items
Processed 180/149823 items
Processed 190/149823 items
Processed 200/149823 items
Processed 210/149823 items
Processed 220/149823 items
Processed 230/149823 items
Processed 240/149823 items
Processed 250/149823 items
Processed 260/149823 items
Processed 270/149823 items
Processed 280/149823 items
Processed 290/149823 items
Processed 300/149823 items
Processed 310/149823 items
Processed 320/149823 items
Processed 330/149823 items
Processed 340/149823 items
Processed 350/149823 items
Processed 360/149823 items
Proces

2. Instalación de LLaVa y dependencias

In [2]:
!git clone https://github.com/haotian-liu/LLaVA.git

Cloning into 'LLaVA'...


remote: Enumerating objects: 2297, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 2297 (delta 2), reused 4 (delta 2), pack-reused 2291
Receiving objects: 100% (2297/2297), 13.71 MiB | 8.40 MiB/s, done.
Resolving deltas: 100% (1402/1402), done.


In [3]:
!cd LLaVA && pip install --upgrade pip && pip install -e .
!cd LLaVA && pip install -e ".[train]"
!pip install flash-attn --no-build-isolation
!pip install deepspeed
!pip install wandb

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.5 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/javiermo/javiersg/LLaVA
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.7/137.7 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

3. Finetuning

In [4]:
# Instalar DeepSpeed y otras dependencias
!pip install deepspeed
!pip install wandb
!pip install flash-attn --no-build-isolation


Defaulting to user installation because normal site-packages is not writeable
  Using cached deepspeed-0.14.4.tar.gz (1.3 MB)
  Preparing metadata (setup.py) ... error
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "/tmp/pip-install-4rbnag4y/deepspeed_c14feef0c9d64ba69c04eb674df4a7f1/setup.py", line 101, in <module>
          cuda_major_ver, cuda_minor_ver = installed_cuda_version()
        File "/tmp/pip-install-4rbnag4y/deepspeed_c14feef0c9d64ba69c04eb674df4a7f1/op_builder/builder.py", line 50, in installed_cuda_version
          raise MissingCUDAException("CUDA_HOME does not exist, unable to compile CUDA op(s)")
      op_builder.builder.MissingCUDAException: CUDA_HOME does not exist, unable to compile CUDA op(s)
      [end of output]
 

In [2]:
import wandb
wandb.login()

!deepspeed LLaVA/llava/train/train_mem.py \
    --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed LLaVA/scripts/zero3.json \
    --model_name_or_path liuhaotian/llava-v1.5-13b \
    --version v1 \
    --data_path ./dataset/train/dataset.json \
    --image_folder ./dataset/train/images \
    --vision_tower openai/clip-vit-large-patch14-336 \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir ./checkpoints/llava-v1.5-13b-task-lora \
    --num_train_epochs 5 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 5000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 10 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jsantamariag (j-santamariag). Use `wandb login --relogin` to force relogin


/bin/bash: line 1: deepspeed: command not found


4. Fusión de pesos

In [3]:
!python LLaVA/scripts/merge_lora_weights.py --model-path checkpoints/llava-v1.5-13b-task-lora --model-base liuhaotian/llava-v1.5-13b --save-model-path llava-ftmodel


/bin/bash: line 1: python: command not found
